In [4]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup

# Cấu hình Moodle API
moodle_url = "http://localhost:8100/webservice/rest/server.php"
api_token = "a5187ae3fec69f55815fa924f038ccf2" #token của webservice moodle

def call_moodle_api(wsfunction, params=None):
    """
    Gọi Moodle Web Service API với hàm và tham số được chỉ định.

    Args:
        wsfunction (str): Tên hàm Web Service.
        params (dict, optional): Tham số bổ sung cho API.

    Returns:
        dict or list: Dữ liệu JSON trả về từ API hoặc None nếu có lỗi.
    """
    default_params = {
        'wstoken': api_token,
        'wsfunction': wsfunction,
        'moodlewsrestformat': 'json'
    }
    if params:
        default_params.update(params)
    
    try:
        response = requests.get(moodle_url, params=default_params)
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            print("Nội dung phản hồi:", response.text)
            return None
        
        data = response.json()
        if isinstance(data, dict) and 'exception' in data:
            print(f"❌ Lỗi API: {data['errorcode']}")
            print(f"Thông điệp: {data['message']}")
            return None
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gửi yêu cầu: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Lỗi phân tích JSON: {e}")
        print("Nội dung phản hồi:", response.text)
        return None


In [5]:
import json

def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học, bao gồm section và module.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_course_get_contents', {'courseid': courseid})

courseid = 5
data = get_course_contents(courseid)  # Giả sử hàm này đã trả về JSON từ Moodle
# Lưu vào file
with open("json_course_moodle.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Đã lưu JSON vào file 'json_course_moodle.json'")

✅ Đã lưu JSON vào file 'json_course_moodle.json'


In [6]:
import json

def build_hierarchical_structure(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        course_data = json.load(f)

    course_structure = []

    def process_modules(modules):
        items = []
        for m in modules:
            mod_info = {
                'id': m.get('id'),
                'name': m.get('name'),
                'modname': m.get('modname')
            }
            items.append(mod_info)
        return items

    def process_sections(sections):
        hierarchy = []
        for sec in sections:
            sec_dict = {
                'sectionid': sec.get('id'),
                'name': sec.get('name'),
                'lessons': []
            }
            for mod in sec.get('modules', []):
                if mod.get('modname') == 'subsection':
                    lesson_dict = {
                        'sectionid': mod.get('id'),
                        'name': mod.get('name'),
                        'resources': process_modules(mod.get('modules', []))  # resource bên trong
                    }
                    sec_dict['lessons'].append(lesson_dict)
                else:
                    # Nếu trực tiếp là tài nguyên trong section
                    sec_dict.setdefault('resources', []).append({
                        'id': mod.get('id'),
                        'name': mod.get('name'),
                        'modname': mod.get('modname')
                    })
            hierarchy.append(sec_dict)
        return hierarchy

    course_structure = process_sections(course_data)
    return course_structure

# --- Demo sử dụng ---
course_hierarchy = build_hierarchical_structure('json_course_moodle.json')
# Lưu vào file
with open("json_course_moodle_simple.json", "w", encoding="utf-8") as f:
    json.dump(course_hierarchy, f, indent=2, ensure_ascii=False)

print("✅ Đã lưu JSON vào file 'json_course_moodle_simple.json'")
print(json.dumps(course_hierarchy, indent=2, ensure_ascii=False))

✅ Đã lưu JSON vào file 'json_course_moodle_simple.json'
[
  {
    "sectionid": 33,
    "name": "General",
    "lessons": [],
    "resources": [
      {
        "id": 52,
        "name": "Announcements",
        "modname": "forum"
      },
      {
        "id": 60,
        "name": "Tin 12 - Định hướng Khoa học Máy tính course question bank",
        "modname": "qbank"
      },
      {
        "id": 102,
        "name": "Bài kiểm tra cuối kỳ môn Tin học - 120825 - 10",
        "modname": "quiz"
      },
      {
        "id": 103,
        "name": "Bài kiểm tra cuối kỳ môn Tin học - 120825 - 11",
        "modname": "quiz"
      },
      {
        "id": 104,
        "name": "Bài kiểm tra cuối kỳ môn Tin học - 120825 - 20",
        "modname": "quiz"
      },
      {
        "id": 105,
        "name": "Bài kiểm tra cuối kỳ môn Tin học - 120825 - 200",
        "modname": "quiz"
      },
      {
        "id": 119,
        "name": "user log viewer",
        "modname": "lti"
      }
    ]
  },
  

In [7]:
import json

# Load JSON gốc
with open("json_course_moodle_simple.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Tạo map từ tên bài học sang resources thực tế
resources_map = {}
for section in data:
    if section.get("resources"):
        resources_map[section["name"]] = {
            "sectionIdNew": section["sectionid"],
            "resources": section["resources"]
        }

new_structure = []

for idx, section in enumerate(data):
    if idx == 0:
        # Giữ nguyên General như cũ
        new_structure.append(section)
        continue

    if section.get("lessons"):  # section có lessons => chủ đề
        new_section = {
            "sectionIdOld": section["sectionid"],
            "name": section["name"],
            "lessons": []
        }

        for lesson in section["lessons"]:
            res_info = resources_map.get(lesson["name"], {"sectionIdNew": lesson["sectionid"], "resources": []})
            new_section["lessons"].append({
                "sectionIdOld": lesson["sectionid"],
                "sectionIdNew": res_info["sectionIdNew"],
                "name": lesson["name"],
                "resources": res_info["resources"]
            })

        new_structure.append(new_section)

    # Bỏ các section riêng lẻ không thuộc chủ đề, ngoại trừ General

# Lưu JSON mới
with open("json_course_moodle_hierarchy_clean.json", "w", encoding="utf-8") as f:
    json.dump(new_structure, f, ensure_ascii=False, indent=2)

print("Đã tạo xong json_course_moodle_hierarchy_clean.json, giữ lại General, bỏ các section riêng lẻ cũ khác.")

Đã tạo xong json_course_moodle_hierarchy_clean.json, giữ lại General, bỏ các section riêng lẻ cũ khác.


In [8]:
# --- Hàm gọi API Moodle ---
def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học, bao gồm section và module.
    Args:
        courseid (int): ID của khóa học.
    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_course_get_contents', {'courseid': courseid})


# --- Hàm xây dựng cấu trúc phân cấp ---
def build_hierarchical_structure(course_data):
    def process_modules(modules):
        items = []
        for m in modules:
            mod_info = {
                'id': m.get('id'),
                'name': m.get('name'),
                'modname': m.get('modname')
            }
            items.append(mod_info)
        return items

    def process_sections(sections):
        hierarchy = []
        for sec in sections:
            sec_dict = {
                'sectionid': sec.get('id'),
                'name': sec.get('name'),
                'lessons': []
            }
            for mod in sec.get('modules', []):
                if mod.get('modname') == 'subsection':
                    lesson_dict = {
                        'sectionid': mod.get('id'),
                        'name': mod.get('name'),
                        'resources': process_modules(mod.get('modules', []))
                    }
                    sec_dict['lessons'].append(lesson_dict)
                else:
                    sec_dict.setdefault('resources', []).append({
                        'id': mod.get('id'),
                        'name': mod.get('name'),
                        'modname': mod.get('modname')
                    })
            hierarchy.append(sec_dict)
        return hierarchy

    return process_sections(course_data)


# --- Hàm làm sạch & tái cấu trúc ---
def clean_structure(hierarchy):
    # Tạo map từ tên section sang resources
    resources_map = {}
    for section in hierarchy:
        if section.get("resources"):
            resources_map[section["name"]] = {
                "sectionIdNew": section["sectionid"],
                "resources": section["resources"]
            }

    new_structure = []

    for idx, section in enumerate(hierarchy):
        if idx == 0:
            # Giữ nguyên General như cũ
            new_structure.append(section)
            continue

        if section.get("lessons"):  # section có lessons => chủ đề
            new_section = {
                "sectionIdOld": section["sectionid"],
                "name": section["name"],
                "lessons": []
            }

            for lesson in section["lessons"]:
                res_info = resources_map.get(
                    lesson["name"],
                    {"sectionIdNew": lesson["sectionid"], "resources": []}
                )
                new_section["lessons"].append({
                    "sectionIdOld": lesson["sectionid"],
                    "sectionIdNew": res_info["sectionIdNew"],
                    "name": lesson["name"],
                    "resources": res_info["resources"]
                })

            new_structure.append(new_section)

        # Bỏ các section riêng lẻ không thuộc chủ đề, ngoại trừ General

    return new_structure


# --- Main ---
if __name__ == "__main__":
    courseid = 5

    # B1. Lấy dữ liệu gốc từ Moodle API
    raw_data = get_course_contents(courseid)

    # Lưu file nguyên bản
    with open("json_course_moodle.json", "w", encoding="utf-8") as f:
        json.dump(raw_data, f, indent=2, ensure_ascii=False)
    print("✅ Đã lưu file nguyên bản: json_course_moodle.json")

    # B2. Xử lý phân cấp
    hierarchy = build_hierarchical_structure(raw_data)

    # B3. Làm sạch dữ liệu (chỉ giữ General + các chủ đề có lessons)
    clean_data = clean_structure(hierarchy)

    # Lưu file kết quả
    with open("json_course_moodle_hierarchy_clean.json", "w", encoding="utf-8") as f:
        json.dump(clean_data, f, indent=2, ensure_ascii=False)
    print("✅ Đã lưu file kết quả: json_course_moodle_hierarchy_clean.json")

✅ Đã lưu file nguyên bản: json_course_moodle.json
✅ Đã lưu file kết quả: json_course_moodle_hierarchy_clean.json
